In [19]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, Transformer

In [2]:
# Lista com os anos dos arquivos
lista_anos = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
dataframes = {}

# Percorrendo a lista e carregando os DataFrames
for ano in lista_anos:
    arquivo = f'../dados/ocorrencias/SIRGAS_SHP_riscoocorrencia_{ano}.shp'
    dataframes[f'df_{ano}'] = gpd.read_file(arquivo)

df_2013 = dataframes['df_2013']
df_2014 = dataframes['df_2014']
df_2015 = dataframes['df_2015']
df_2016 = dataframes['df_2016']
df_2017 = dataframes['df_2017']
df_2018 = dataframes['df_2018']
df_2019 = dataframes['df_2019']
df_2020 = dataframes['df_2020']
df_2021 = dataframes['df_2021']
df_2022 = dataframes['df_2022']
df_2023 = dataframes['df_2023']
df_2024 = dataframes['df_2024']

In [21]:
df_total = gpd.GeoDataFrame(pd.concat([df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True))

In [22]:
df_total

,data,ocorrencia,subprefeit,name,geometry,subpreit
0,2013-12-29 00:00:00,QUEDA DE ARVORE,BT - BUTANTA,47368,POINT Z (318923.714 7389107.424 0),NaN
1,2013-12-03 00:00:00,QUEDA DE ARVORE,CS - CAPELA DO SOCORRO,45006,POINT Z (326291.01 7378438.554 0),NaN
2,2013-12-10 00:00:00,ALAGAMENTO,SA - SANTO AMARO,45361,POINT Z (325784.879 7383070.03 0),NaN
3,2013-12-30 00:00:00,QUEDA DE ARVORE,SE - SE,47427,POINT Z (332913.137 7396238.346 0),NaN
4,2013-12-29 00:00:00,QUEDA DE ARVORE,SA - SANTO AMARO,47337,POINT Z (325940.608 7385402.683 0),NaN
...,...,...,...,...,...,...
58147,2024-01-31 00:00:00,ALAGAMENTO,NaN,32057914,POINT (347483.632 7386553.452),SB - SAPOPEMBA
58148,2024-01-31 00:00:00,QUEDA DE ARVORE,NaN,32058056,POINT (354127.126 7394828.661),IQ - ITAQUERA
58149,2024-01-31 00:00:00,INUNDACAO,NaN,32058336,POINT (328375.103 7375680.959),CS - CAPELA DO SOCORRO
58150,2024-01-31 00:00:00,QUEDA DE ARVORE,NaN,32058368,POINT (324562.933 7390714.965),BT - BUTANTA


In [23]:
df_total1 = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)
df_total2 = pd.concat([df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020], ignore_index=True)

df_total1 = df_total1.rename(columns={'subpreit':'subprefeit'})

df_total = pd.concat([df_total2, df_total1], ignore_index=True)

In [24]:
df_total

,data,ocorrencia,subprefeit,name,geometry
0,2013-12-29 00:00:00,QUEDA DE ARVORE,BT - BUTANTA,47368,POINT Z (318923.714 7389107.424 0)
1,2013-12-03 00:00:00,QUEDA DE ARVORE,CS - CAPELA DO SOCORRO,45006,POINT Z (326291.01 7378438.554 0)
2,2013-12-10 00:00:00,ALAGAMENTO,SA - SANTO AMARO,45361,POINT Z (325784.879 7383070.03 0)
3,2013-12-30 00:00:00,QUEDA DE ARVORE,SE - SE,47427,POINT Z (332913.137 7396238.346 0)
4,2013-12-29 00:00:00,QUEDA DE ARVORE,SA - SANTO AMARO,47337,POINT Z (325940.608 7385402.683 0)
...,...,...,...,...,...
58147,2024-01-31 00:00:00,ALAGAMENTO,SB - SAPOPEMBA,32057914,POINT (347483.632 7386553.452)
58148,2024-01-31 00:00:00,QUEDA DE ARVORE,IQ - ITAQUERA,32058056,POINT (354127.126 7394828.661)
58149,2024-01-31 00:00:00,INUNDACAO,CS - CAPELA DO SOCORRO,32058336,POINT (328375.103 7375680.959)
58150,2024-01-31 00:00:00,QUEDA DE ARVORE,BT - BUTANTA,32058368,POINT (324562.933 7390714.965)


In [25]:
df_total.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 58152 entries, 0 to 58151
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   data        58152 non-null  object  
 1   ocorrencia  58152 non-null  object  
 2   subprefeit  58151 non-null  object  
 3   name        58152 non-null  object  
 4   geometry    58152 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 2.2+ MB


In [26]:
df_total['geometry'] = df_total['geometry'].astype(str)

C:\Users\prado\AppData\Local\Temp\ipykernel_45416\883087392.py:1: UserWarning: Geometry column does not contain geometry.
  df_total['geometry'] = df_total['geometry'].astype(str)


In [27]:
df_total.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 58152 entries, 0 to 58151
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   data        58152 non-null  object
 1   ocorrencia  58152 non-null  object
 2   subprefeit  58151 non-null  object
 3   name        58152 non-null  object
 4   geometry    58152 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [35]:
# Definindo o sistema de coordenadas UTM zona 23S
utm_proj_2d = Proj(proj='utm', zone=23, south=True, ellps='WGS84')
utm_proj_3d = Proj(proj='utm', zone=23, south=True, ellps='WGS84', datum='WGS84')
wgs84_proj = Proj(proj='latlong', datum='WGS84')

transformer_2d = Transformer.from_proj(utm_proj_2d, wgs84_proj)
transformer_3d = Transformer.from_proj(utm_proj_3d, wgs84_proj)

def converte_para_latlon(point_str):
    # Detectar se é POINT Z ou POINT
    if 'POINT Z' in point_str:
        # Extrair as coordenadas do formato POINT Z
        parts = point_str.replace('POINT Z (', '').replace(')', '').split()
        x, y = float(parts[0]), float(parts[1])
        # Converter coordenadas UTM 3D para latitude e longitude
        lat, lon = transformer_3d.transform(x, y)
    elif 'POINT' in point_str:
        # Extrair as coordenadas do formato POINT
        parts = point_str.replace('POINT (', '').replace(')', '').split()
        x, y = float(parts[0]), float(parts[1])
        # Converter coordenadas UTM 2D para latitude e longitude
        lat, lon = transformer_2d.transform(x, y)
    else:
        # Caso não seja um formato reconhecido
        return None, None
    return lat, lon

# Aplicar a função para criar as novas colunas 'Latitude' e 'Longitude'
df_total[['Longitude', 'Latitude']] = df_total['geometry'].apply(lambda x: pd.Series(converte_para_latlon(x)))

# Exibindo o DataFrame atualizado
df_total

,data,ocorrencia,subprefeit,name,geometry,Latitude,Longitude
0,2013-12-29 00:00:00,QUEDA DE ARVORE,BT - BUTANTA,47368,POINT Z (318923.7137 7389107.424428 0),-23.598443,-46.774643
1,2013-12-03 00:00:00,QUEDA DE ARVORE,CS - CAPELA DO SOCORRO,45006,POINT Z (326291.010146 7378438.554415 0),-23.695577,-46.703712
2,2013-12-10 00:00:00,ALAGAMENTO,SA - SANTO AMARO,45361,POINT Z (325784.879025 7383070.030225 0),-23.653706,-46.708130
3,2013-12-30 00:00:00,QUEDA DE ARVORE,SE - SE,47427,POINT Z (332913.137082 7396238.345912 0),-23.535563,-46.636786
4,2013-12-29 00:00:00,QUEDA DE ARVORE,SA - SANTO AMARO,47337,POINT Z (325940.608356 7385402.683087 0),-23.632662,-46.706331
...,...,...,...,...,...,...,...
58147,2024-01-31 00:00:00,ALAGAMENTO,SB - SAPOPEMBA,32057914,POINT (347483.631646 7386553.451906),-23.624450,-46.495083
58148,2024-01-31 00:00:00,QUEDA DE ARVORE,IQ - ITAQUERA,32058056,POINT (354127.125688 7394828.6609),-23.550340,-46.429165
58149,2024-01-31 00:00:00,INUNDACAO,CS - CAPELA DO SOCORRO,32058336,POINT (328375.102682 7375680.958842),-23.720698,-46.683598
58150,2024-01-31 00:00:00,QUEDA DE ARVORE,BT - BUTANTA,32058368,POINT (324562.933229 7390714.964635),-23.584550,-46.719206


In [36]:
df_total.to_csv('../dados/ocorrencias_2013_2024.csv', index=False)